In [72]:
import sys
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [73]:
survey = spark.read \
  .option('header', True) \
  .csv('data/2016-stack-overflow-survey-responses.csv')

In [74]:
count_all = spark.sparkContext.accumulator(0)
count_canada = spark.sparkContext.accumulator(0)
count_missing_salary_midpoint = spark.sparkContext.accumulator(0)
total_bytes = spark.sparkContext.accumulator(0)

In [75]:
def summarize(s):
  for result in s:
    count_all.add(1)
    total_bytes.add(sys.getsizeof(result))
    if not result.salary_midpoint:
      count_missing_salary_midpoint.add(1)
    if result.country.upper() == 'CANADA':
      count_canada.add(1)
      
survey.foreachPartition(summarize)

In [76]:
print('Total count: {}'.format(count_all.value))
print('Canada count: {}'.format(count_canada.value))
print('Missing salary midpoint: {}'.format(count_missing_salary_midpoint.value))
print('Total bytes: {}'.format(total_bytes.value))

Total count: 1999
Canada count: 77
Missing salary midpoint: 566
Total bytes: 1167416
